In [1]:
import pandas as pd
import folium
import re
import googlemaps
import ast
import time

In [2]:
df = pd.read_csv("youtube_videos.csv")

In [3]:
with open("youtube_api_key.txt","r") as file:
    api_key = file.read().strip()

In [4]:
import requests
api_response = requests.get("https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8")

In [5]:
api_response.url

'https://www.google.com/maps/place/Kari+dosa/@12.9136665,77.6026948,17z/data=!4m6!3m5!1s0x3bae1500439dae5d:0xff8fc22bcc73fc25!8m2!3d12.9136665!4d77.6052697!16s%2Fg%2F11vs35r34t?entry=tts'

In [6]:
# Regular expression to find latitude and longitude
pattern1 = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
pattern2 = r'%40(-?\d+\.\d+),(-?\d+\.\d+)'

# Search for the pattern in the URL
match = re.search(pattern1, api_response.url)

if match:
    latitude = match.group(1)
    longitude = match.group(2)
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("No latitude and longitude found.")

Latitude: 12.9136665, Longitude: 77.6026948


In [7]:
def extract_lat_long_from_dynamic_url(short_url):
    try:
        response = requests.get(short_url)
        final_url = response.url
        # Regular expressions to find latitude and longitude in different formats
        pattern1 = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
        pattern2 = r'%40(-?\d+\.\d+),(-?\d+\.\d+)'
        pattern3 = r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)'
        # Search for the patterns in the URL
        match1 = re.search(pattern1, final_url)
        match2 = re.search(pattern2, final_url)
        match3 = re.search(pattern3, final_url)
        if match1:
            latitude = match1.group(1)
            longitude = match1.group(2)
            return latitude, longitude
        elif match2:
            latitude = match2.group(1)
            longitude = match2.group(2)
            return latitude, longitude
        elif match3:
            latitude = match3.group(1)
            longitude = match3.group(2)
            return latitude, longitude
        else:
            return None, None
    except requests.RequestException as e:
        print(f"Failed to fetch URL: {short_url}, error: {e}")
        return None, None

In [8]:
df.head()

,Title,Link,Description,Links,gmaps_links,video_id,transcript
0,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,['https://youtu.be/m2G-fnkoaOY'],NaN,BfP0-J08OoY,to us Biryani is not just meat and rice and sp...
1,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"['https://youtu.be/Q-jNfDaYuMc', 'https://maps...",['https://maps.app.goo.gl/2X6idFktNJv8D4CW8'],soAOqI-dCmM,Time for Lunch Butt I Feel Like Eating Light a...
2,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,['https://www.youtube.com/c/KripalAmannaVlogs'...,['https://maps.app.goo.gl/2X6idFktNJv8D4CW8'],Q-jNfDaYuMc,"Error: the JSON object must be str, bytes or b..."
3,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,['https://www.youtube.com/c/KripalAmannaVlogs'...,['https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96'],eMgRwe1vePg,"Error: the JSON object must be str, bytes or b..."
4,Shining Some Light On The Unsung Eateries Of B...,https://www.youtube.com/watch?v=4KzC3QQkyts,Namaskara folks! Hoping to shine a bit of ligh...,[],NaN,4KzC3QQkyts,namaskara folks this is kpal gour on the road ...


In [9]:
links = df[df["gmaps_links"].notnull()][["gmaps_links","Link","Title","Description","transcript"]]

In [10]:
links.info()

<class 'pandas.core.frame.DataFrame'>
Index: 599 entries, 1 to 747
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gmaps_links  599 non-null    object
 1   Link         599 non-null    object
 2   Title        599 non-null    object
 3   Description  599 non-null    object
 4   transcript   599 non-null    object
dtypes: object(5)
memory usage: 28.1+ KB


In [11]:
def evaluate_list(string):
    return ast.literal_eval(string)
links["gmaps_links"] = links["gmaps_links"].apply(evaluate_list)

In [12]:
links.head()

,gmaps_links,Link,Title,Description,transcript
1,[https://maps.app.goo.gl/2X6idFktNJv8D4CW8],https://www.youtube.com/watch?v=soAOqI-dCmM,Delicious Must-Try Millet Dishes At This Small...,From millet offerings in pongals to bisi bele ...,Time for Lunch Butt I Feel Like Eating Light a...
2,[https://maps.app.goo.gl/2X6idFktNJv8D4CW8],https://www.youtube.com/watch?v=Q-jNfDaYuMc,She Wants To Serve Healthy Traditional Tiffin ...,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,"Error: the JSON object must be str, bytes or b..."
3,[https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96],https://www.youtube.com/watch?v=eMgRwe1vePg,"This Family Brings ""Healthier No Maida Pizza"" ...",#hotelgoldenriviera #pizza #mysore\n \nherbs o...,"Error: the JSON object must be str, bytes or b..."
7,[https://maps.app.goo.gl/5bcyGF2Xys5U8cJH6],https://www.youtube.com/watch?v=HG9JmewVE04,"Lunch At A Highway Eatery! Namma Hallimane, By...","Only about three years old, however Namma Hall...",M Kaa M Kaa M [Music] mut Chicken Chops one by...
8,"[https://maps.app.goo.gl/aH7rvnS5itWGF2dy6, ht...",https://www.youtube.com/watch?v=m2G-fnkoaOY,"Tasting Bengaluru’s Oldest, Most Popular Birya...",#tajhotelbiryani #shivajimilitaryhotelbiryani ...,so the third Biryani that we about to taste is...


In [13]:
links["gmaps_links"].apply(len).value_counts()

gmaps_links
1    545
2     32
3      8
5      6
4      3
7      2
6      2
8      1
Name: count, dtype: int64

In [14]:
for sublist in links["gmaps_links"]:
    for item in sublist:
        print(item)
        break

https://maps.app.goo.gl/2X6idFktNJv8D4CW8
https://maps.app.goo.gl/2X6idFktNJv8D4CW8
https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96
https://maps.app.goo.gl/5bcyGF2Xys5U8cJH6
https://maps.app.goo.gl/aH7rvnS5itWGF2dy6
https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8
https://maps.app.goo.gl/CzFaV8qf3YXkPczu9
https://goo.gl/maps/kqUtDU14PkpJV4gT6
https://goo.gl/maps/48tg1bR4n3swbzGXA
https://maps.app.goo.gl/CzFaV8qf3YXkPczu9
https://maps.app.goo.gl/mvvkuqfdYxsCjD8t9
https://maps.app.goo.gl/gGSmVCU9w5efBReT8
https://maps.app.goo.gl/8Txm7yqFt3JX81469
https://maps.app.goo.gl/mvvkuqfdYxsCjD8t9
https://maps.app.goo.gl/uKGRoe6u3h8SNZ1J7
https://maps.app.goo.gl/dcaGdjFmb2Apfnwz7
https://maps.google.com/?q=11.785901,78.208954
https://goo.gl/maps/zrCuARY8qXzBJM4q6
https://maps.app.goo.gl/GmqsyKnwDCw78uZ37
https://goo.gl/maps/bZat92ZaSkeRCVyJA
https://maps.app.goo.gl/yTMfQKW3SrZSyFom8
https://maps.app.goo.gl/YohtprzBHRedfDiz5
https://maps.app.goo.gl/rN6vZH8n1DEMy9Aw7
https://maps.app.goo.gl/rN6vZH8n1DEMy9Aw7
htt

In [15]:
# Initialize an empty dictionary and an empty list
gmaps_to_link = {}
all_gmaps_links = []

# Iterate over the rows of the DataFrame
for index, row in links.iterrows():
    # Get the value from the 'Link' column
    link = row['Link']
    description = row['Description']
    transcript = row['transcript']
    title = row['Title']
    
    # Get the corresponding Google Maps links from the 'gmaps_links' column
    gmaps_links = row['gmaps_links']
    
    # Iterate through each Google Maps link
    for gmaps_link in gmaps_links:
        # Add the Google Maps link to the all_gmaps_links list
        all_gmaps_links.append(gmaps_link)
        
        if gmaps_link not in gmaps_to_link:
            gmaps_to_link[gmaps_link] = [{
                'title': title,
                'link': link,
                'description': description,
                'transcription': transcript
            }]
        else:
            # If the Google Maps link is already in the dictionary, append the new details
            gmaps_to_link[gmaps_link].append({
                'title': title,
                'link': link,
                'description': description,
                'transcription': transcript
            })


# Optional: Remove duplicates from the all_gmaps_links list if needed
all_gmaps_links = list(set(all_gmaps_links))

# gmaps_to_link, all_gmaps_links

In [16]:
# gmaps_to_link

In [17]:
all_links = [link for sublist in links['gmaps_links'] for link in sublist]
all_links[:5]

['https://maps.app.goo.gl/2X6idFktNJv8D4CW8',
 'https://maps.app.goo.gl/2X6idFktNJv8D4CW8',
 'https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96',
 'https://maps.app.goo.gl/5bcyGF2Xys5U8cJH6',
 'https://maps.app.goo.gl/aH7rvnS5itWGF2dy6']

In [18]:
len(gmaps_to_link)

512

In [19]:
# link_df = pd.DataFrame(all_links, columns=["link"])
# link_df.head()

In [20]:
# link_df[['latitude', 'longitude']] = link_df['link'].apply(lambda x: pd.Series(extract_lat_long_from_dynamic_url(x)))

In [21]:
link_df = pd.read_csv("extracted_lat_long.csv")
link_df.head()

,link,latitude,longitude
0,https://maps.app.goo.gl/5bcyGF2Xys5U8cJH6,12.958374,75.979502
1,https://maps.app.goo.gl/aH7rvnS5itWGF2dy6,12.985202,77.604485
2,https://goo.gl/maps/mZqgz8ZNB4C8e5s38,12.918249,77.573504
3,https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8,12.913666,77.602695
4,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9,NaN,NaN


In [23]:
link_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   link       706 non-null    object 
 1   latitude   525 non-null    float64
 2   longitude  525 non-null    float64
dtypes: float64(2), object(1)
memory usage: 16.7+ KB


In [24]:
link_df[link_df["latitude"].isnull()]["link"].nunique()

151

In [25]:
def extract_lat_long_from_google_maps_url(url):
    try:
        # Regular expression to find latitude and longitude
        pattern = r'\?q=(-?\d+\.\d+),(-?\d+\.\d+)'
        # Search for the pattern in the URL
        match = re.search(pattern, url)
        if match:
            latitude = match.group(1)
            longitude = match.group(2)
            return latitude, longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error occurred while extracting latitude and longitude: {e}")
        return None, None

In [26]:
# Apply the function to the DataFrame where latitude is null
for idx, row in link_df[link_df['latitude'].isnull()].iterrows():
    lat, long = extract_lat_long_from_google_maps_url(row['link'])
    link_df.at[idx, 'latitude'] = lat
    link_df.at[idx, 'longitude'] = long

In [27]:
link_df[link_df["latitude"].isnull()][["link"]]

,link
4,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9
8,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9
14,https://maps.app.goo.gl/dcaGdjFmb2Apfnwz7
24,https://maps.app.goo.gl/YohtprzBHRedfDiz5
90,https://g.page/SandeepsRosomelette?share
...,...
701,https://maps.app.goo.gl/7BsHd7psfbHtcnoN6
702,https://maps.app.goo.gl/24YwTcG8qpEhM15FA
703,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7
704,https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7


In [28]:
link_df.head()

,link,latitude,longitude
0,https://maps.app.goo.gl/5bcyGF2Xys5U8cJH6,12.958374,75.979502
1,https://maps.app.goo.gl/aH7rvnS5itWGF2dy6,12.985202,77.604485
2,https://goo.gl/maps/mZqgz8ZNB4C8e5s38,12.918249,77.573504
3,https://maps.app.goo.gl/Pwu2EkJbnj59r3wV8,12.913666,77.602695
4,https://maps.app.goo.gl/CzFaV8qf3YXkPczu9,NaN,NaN


In [29]:
link_df.columns

Index(['link', 'latitude', 'longitude'], dtype='object')

In [30]:
for index, row in link_df.iterrows():
    gmaps_link = row['link']
    latitude = row['latitude']
    longitude = row['longitude']
    if gmaps_link in gmaps_to_link:
        # Iterate through each entry in the list for this gmaps_link
        for entry in gmaps_to_link[gmaps_link]:
            entry['lat'] = latitude
            entry['long'] = longitude

In [31]:
# gmaps_to_link

In [32]:
# import json

# # Assuming gmaps_to_link dictionary is already created as shown in the previous example

# # Define the filename for the JSON file
# json_filename = 'gmaps_to_link.json'

# # Dump the dictionary into the JSON file
# with open(json_filename, 'w') as json_file:
#     json.dump(gmaps_to_link, json_file, indent=4)

# print(f"Dictionary has been dumped into {json_filename}")

In [33]:
def print_tree(d, indent=0):
    for key, value in d.items():
        print('  ' * indent + str(key))
        if isinstance(value, dict):
            print_tree(value, indent+1)
        else:
            print('  ' * (indent+1) + str(value))

In [35]:
# print_tree(gmaps_to_link)

In [35]:
api_response = requests.get("https://maps.app.goo.gl/3sKkHUEFtfT8ENzr7")
api_response.url

'https://www.google.com/maps/place//data=!4m2!3m1!1s0x3ba35a4bd663a737:0x947fd736e73ae5cc?utm_source=mstt_1'

In [36]:
api_response = requests.get("https://g.page/SandeepsRosomelette?share")
api_response.url

'https://www.google.com/search?q=SandeepsRosomelette&ludocid=15119326801541647727&source=g.page.share'

In [37]:
link_df.to_csv("extracted_lat_long.csv", index=False)

In [38]:
link_df.columns

Index(['link', 'latitude', 'longitude'], dtype='object')

In [39]:
df.head()

,Title,Link,Description,Links,gmaps_links,video_id,transcript
0,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,['https://youtu.be/m2G-fnkoaOY'],NaN,BfP0-J08OoY,to us Biryani is not just meat and rice and sp...
1,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"['https://youtu.be/Q-jNfDaYuMc', 'https://maps...",['https://maps.app.goo.gl/2X6idFktNJv8D4CW8'],soAOqI-dCmM,Time for Lunch Butt I Feel Like Eating Light a...
2,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,['https://www.youtube.com/c/KripalAmannaVlogs'...,['https://maps.app.goo.gl/2X6idFktNJv8D4CW8'],Q-jNfDaYuMc,"Error: the JSON object must be str, bytes or b..."
3,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,['https://www.youtube.com/c/KripalAmannaVlogs'...,['https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96'],eMgRwe1vePg,"Error: the JSON object must be str, bytes or b..."
4,Shining Some Light On The Unsung Eateries Of B...,https://www.youtube.com/watch?v=4KzC3QQkyts,Namaskara folks! Hoping to shine a bit of ligh...,[],NaN,4KzC3QQkyts,namaskara folks this is kpal gour on the road ...


In [40]:
import json

with open("gmaps_to_link.json", "r") as file:
    gmaps_to_link = json.load(file)

In [41]:
# gmaps_to_link

In [44]:
import folium

m = folium.Map(location=[12.9, 77.5], zoom_start=7)

# Add markers for each point in the dictionary
for link, info_list in gmaps_to_link.items():
    for info in info_list:
        if not pd.isnull(info["lat"]) and not pd.isnull(info["long"]):
            folium.Marker(
                location=[info["lat"], info["long"]],
                popup=f"<b>{info['title']}</b><br>Youtube: <a href='{info['link']}' target='_blank'>Link</a>",
                tooltip=info["title"]
            ).add_to(m)


# Display the map
m

In [45]:
m.save("youtube_map_2.html")

In [46]:
m = folium.Map(location=[12.9, 77.5], zoom_start=7, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}', attr='Esri')

# Add markers for each point in the dictionary
for link, info_list in gmaps_to_link.items():
    for info in info_list:
        if not pd.isnull(info["lat"]) and not pd.isnull(info["long"]):
            popup_content = f"<b>{info['title']}</b><br>"
            if 'link' in info:
                popup_content += f"YouTube: <a href='{info['link']}' target='_blank'><i class='fab fa-youtube'></i></a><br>"
            # if 'gmaps_link' in info:
            #     popup_content += f"Google Maps: <a href='{info['gmaps_link']}' target='_blank'><i class='fab fa-google'></i></a>"
                
            folium.Marker(
                location=[info["lat"], info["long"]],
                popup=folium.Popup(popup_content,max_width='50%'),
                tooltip=info["title"]
            ).add_to(m)
m

In [47]:
m.save("Website/youtube_map_3.html")